In [1]:
import boto3
import botocore
import torchdata.datapipes as dp
from pathlib import Path
import shutil
from src.globals import DATA_DIR
import scipy
import skimage
import tifffile
from joblib import Parallel, delayed
import os
import json
import pandas as pd
import tqdm

/home/philip/miniconda3/envs/mi4p/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.DataFrame()
l = []
for j in tqdm.tqdm(
    os.listdir(
        "/home/philip/Projects/MI4People/MI4People_soil_quality/data/raw/bigearth_labels"
    )
):
    with open(
        os.path.join(
            "/home/philip/Projects/MI4People/MI4People_soil_quality/data/raw/bigearth_labels",
            j,
        ),
        "r",
    ) as f:
        json_content = json.load(f)
        # print(json_content)
        l.append(json_content)
    # break

100%|██████████| 590326/590326 [02:10<00:00, 4511.16it/s]


In [9]:
df = pd.DataFrame(l)
df[["ulx", "uly", "lrx", "lry"]] = pd.json_normalize(df["coordinates"])

In [4]:
df.head()

,labels,coordinates,projection,tile_source,acquisition_date,ulx,uly,lrx,lry
0,"[Coniferous forest, Mixed forest, Transitional...","{'ulx': 619200, 'uly': 7330440, 'lrx': 620400,...","PROJCS[""WGS 84 / UTM zone 35N"",GEOGCS[""WGS 84""...",S2A_MSIL1C_20180219T094031_N0206_R036_T35WPP_2...,2018-02-19 09:40:31,619200,7330440,620400,7329240
1,[Coniferous forest],"{'ulx': 532380, 'uly': 5273640, 'lrx': 533580,...","PROJCS[""WGS 84 / UTM zone 33N"",GEOGCS[""WGS 84""...",S2B_MSIL1C_20180421T100029_N0206_R122_T33TWN_2...,2018-04-21 10:00:29,532380,5273640,533580,5272440
2,"[Coniferous forest, Mixed forest]","{'ulx': 593580, 'uly': 7040040, 'lrx': 594780,...","PROJCS[""WGS 84 / UTM zone 35N"",GEOGCS[""WGS 84""...",S2A_MSIL1C_20170905T095031_N0205_R079_T35VNL_2...,2017-09-05 09:50:31,593580,7040040,594780,7038840
3,"[Non-irrigated arable land, Pastures]","{'ulx': 666000, 'uly': 5829240, 'lrx': 667200,...","PROJCS[""WGS 84 / UTM zone 29N"",GEOGCS[""WGS 84""...",S2B_MSIL1C_20171112T114339_N0206_R123_T29UPU_2...,2017-11-12 11:43:39,666000,5829240,667200,5828040
4,[Sea and ocean],"{'ulx': 409560, 'uly': 6672420, 'lrx': 410760,...","PROJCS[""WGS 84 / UTM zone 34N"",GEOGCS[""WGS 84""...",S2B_MSIL1C_20170814T100029_N0205_R122_T34VDM_2...,2017-08-14 10:00:29,409560,6672420,410760,6671220


In [10]:
labels = df.explode("labels")["labels"]

In [45]:
ind_to_label = {i: label for (i, label) in enumerate(labels.unique())}
label_df = pd.DataFrame.from_records([ind_to_label]).T
# label_df = label_df.rename(columns={0: "label"})
label_df.to_json(
    "/home/philip/Projects/MI4People/MI4People_soil_quality/data/raw/bigearth_labels.json"
)
# label_to_ind = pd.DataFrame({label: i for (i, label) in ind_to_label.items()}, index=ind_to_label.keys())
# label_to_ind.to_csv()

In [42]:
grouped = df.groupby(["ulx", "uly", "lrx", "lry"])

In [24]:
from collections import Counter

Counter(grouped.size().values)

Counter({1: 88856, 2: 46809, 4: 66056, 3: 47876})